In [1]:
import itertools
import numpy as np
from sympol import Polytope
from sympol.ehrhart import h_star_vector_of_cartesian_product_from_h_star_vectors, h_to_gamma_vector, gamma_to_h_vector

from sympy import Matrix
from sympy.matrices.normalforms import hermite_normal_form

minimals = [
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1, 1, 1, 2], [-1, -1, -1, -2], [0, 0, -1, 0], [0, -1, 0, 0], [-1, 0, 0, 0]],
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, -1], [0, 0, -1, 0], [0, -1, 0, 0], [-1, 0, 0, 0]],
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 2], [0, 0, -1, -2], [0, 0, -1, 0], [0, -1, 0, 0], [-1, 0, 0, 0]],
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 2, 0], [1, 0, 0, 2], [-1, 0, 0, -2], [0, -1, -2, 0], [0, -1, 0, 0], [-1, 0, 0, 0]],
    [[1, 0, 0, 0], [1, 2, 0, 0], [1, 0, 2, 0], [1, 0, 0, 2], [-1, 0, 0, -2], [-1, 0, -2, 0], [-1, -2, 0, 0], [-1, 0, 0, 0]],
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 2, 0], [0, 1, 0, 2], [0, -1, 0, -2], [0, -1, -2, 0], [0, -1, 0, 0], [-1, 0, 0, 0]],
]

def normal_form(p):
    vertst = tuple(sorted([tuple(v) for v in p.vertices.T]))

    all_vertst = []
    for p in itertools.permutations(vertst):
        permuted_vertst = tuple(sorted(p))
        hnf = hermite_normal_form(Matrix(permuted_vertst))
        all_vertst.append(hnf)
    

    nf = np.array(min(all_vertst))
    return tuple(sorted([tuple(v) for v in nf.T]))


all_cs = dict()

for verts_p in minimals:
    p = Polytope(vertices=verts_p)
    pd = p.dual
    np_pd = pd.n_integer_points
    if np_pd not in all_cs:
        all_cs[np_pd] = set([normal_form(pd)])
    else:
        all_cs[np_pd].add(normal_form(pd))

v = max(all_cs.keys())

while v > 0:
    for verts in all_cs[v]:
        p = Polytope(vertices=verts)
        p_pts = p.integer_points
        for v in verts:
            # avoid removing the same vertices twice
            nzv = [a for a in v if a != 0]
            if nzv[0] < 0:
                continue

            # remove the vertex and its opposite
            v_plus = tuple([a for a in v])
            v_minus = tuple([-a for a in v])
            new_verts = [u for u in p_pts if tuple(u) not in [v_plus, v_minus]]
            assert len(new_verts) == len(p_pts) - 2

            # check if the new polytope is full dimensional and canonical
            q = Polytope(new_verts)
            assert q.is_centrally_symmetric
            if q.dim < p.dim or not q.contains(q._origin(), strict=True):
                continue

            np_q = q.n_integer_points
            if np_q not in all_cs:
                all_cs[np_q] = set([normal_form(q)])
            else:
                all_cs[np_q].add(normal_form(q))

    for np_q, set_verts in all_cs.items():
        print(np_q, len(set_verts))
            
    v -= 1
